In [488]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from sklearn import preprocessing
import keras
from keras import layers,Sequential

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [489]:
data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

In [490]:
test_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [491]:
y_train = data["Survived"]

In [492]:
def clean_data(data,*drop_columns):
    print(drop_columns,"are dropped")
    column_array = []
    for x in drop_columns:
        column_array.append(x)
    drop = data.drop(column_array,axis=1)
    return drop

In [493]:
cleaned_data = clean_data(data,"Survived","PassengerId","Name","Cabin","Ticket")
test_data = clean_data(test_data,"PassengerId","Name","Cabin","Ticket")

('Survived', 'PassengerId', 'Name', 'Cabin', 'Ticket') are dropped
('PassengerId', 'Name', 'Cabin', 'Ticket') are dropped


In [494]:
cleaned_data['Sex'].replace({'male':1,'female':0},inplace=True)
test_data['Sex'].replace({'male':1,'female':0},inplace=True)

In [495]:
cleaned_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,S
1,1,0,38.0,1,0,71.2833,C
2,3,0,26.0,0,0,7.9250,S
3,1,0,35.0,1,0,53.1000,S
4,3,1,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,1,27.0,0,0,13.0000,S
887,1,0,19.0,0,0,30.0000,S
888,3,0,NaN,1,2,23.4500,S
889,1,1,26.0,0,0,30.0000,C


In [496]:
cleaned_data["Embarked"].fillna("E",inplace=True)
columns = ["Age","SibSp","Parch","Fare"]
for col in columns:
    mean_value = cleaned_data[f'{col}'].mean()
    cleaned_data[f'{col}'].fillna(mean_value, inplace=True)
    
    
test_data["Embarked"].fillna("E",inplace=True)
columns = ["Age","SibSp","Parch","Fare"]
for col in columns:
    mean_value = test_data[f'{col}'].mean()
    test_data[f'{col}'].fillna(mean_value, inplace=True)

In [497]:
le = preprocessing.LabelEncoder()
cleaned_data['Embarked'] = le.fit_transform(cleaned_data['Embarked'])
test_data['Embarked'] = le.fit_transform(test_data['Embarked'])

In [498]:
cleaned_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.000000,1,0,7.2500,3
1,1,0,38.000000,1,0,71.2833,0
2,3,0,26.000000,0,0,7.9250,3
3,1,0,35.000000,1,0,53.1000,3
4,3,1,35.000000,0,0,8.0500,3
...,...,...,...,...,...,...,...
886,2,1,27.000000,0,0,13.0000,3
887,1,0,19.000000,0,0,30.0000,3
888,3,0,29.699118,1,2,23.4500,3
889,1,1,26.000000,0,0,30.0000,0


In [499]:
def normalization(cleaned_data,feature):
    mean = cleaned_data[feature].mean()
    cleaned_data[feature] -= mean
    std = cleaned_data[feature].std()
    cleaned_data[feature] /= std

In [500]:
test_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.50000,0,0,7.8292,1
1,3,0,47.00000,1,0,7.0000,2
2,2,1,62.00000,0,0,9.6875,1
3,3,1,27.00000,0,0,8.6625,2
4,3,0,22.00000,1,1,12.2875,2
...,...,...,...,...,...,...,...
413,3,1,30.27259,0,0,8.0500,2
414,1,0,39.00000,0,0,108.9000,0
415,3,1,38.50000,0,0,7.2500,2
416,3,1,30.27259,0,0,8.0500,2


In [501]:
features = ['Pclass','Sex',"Age","SibSp","Parch","Fare","Embarked"]
for x in features:
    normalization(cleaned_data,x)
    normalization(test_data,x)

In [502]:
x_train = cleaned_data.to_numpy()
y_train = y_train.to_numpy()
x_test = test_data.to_numpy()

In [504]:
model = Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(7,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

model.fit(x_train,y_train,epochs=1000,verbose=0)

In [506]:
prediction = model.predict(x_test)

14/14 [==============================] - 0s 1ms/step


In [507]:
prediction = (prediction > 0.5).astype(np.float32)

In [508]:
a = pd.read_csv('/kaggle/input/titanic/test.csv')
final = pd.DataFrame(a["PassengerId"])
final["Survived"] = prediction
final.to_csv('/kaggle/working/final_predictions.csv',index=False)

In [505]:
model.evaluate(x_train,y_train)

28/28 [==============================] - 0s 2ms/step - loss: 0.1250 - acc: 0.9506


[0.12498936802148819, 0.9506173133850098]

Final score = 95.06%